# Multiple Linear Regression

Most variables of interest probably depend upon more than just a single feature. That is why statisticians invented multiple linear regression.

## What we will accomplish

In this notebook we will:
- Introduce the multiple linear regression model,
- Show how to fit the model using the <i>normal equation</i>,
- Fit a sample model with `numpy` and
- Fit the same model with `sklearn`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_style

set_style("whitegrid")

## The multiple linear regression model

Suppose there is a quantitative variable you want to predict/model called $y$ and a set of $m$ features $X_1, X_2, \dots X_m$, then the multiple linear regression model regressing $y$ on $X_1, \dots, X_m$ is:

$$
y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_m X_m + \epsilon = X \beta + \epsilon,
$$

where $\beta_0, \dots, \beta_m \in \mathbb{R}$ are constants, $\beta$ is an $(m+1) \times 1$ vector of the $\beta_i$s in numerical order, 

$$
X = \left(1 | X_1 | X_2 | \dots | X_m \right),
$$

and $\epsilon \sim N(0,\sigma)$ is an error term independent of $X$.

### Fitting the model

Suppose that we have $n$ observations $(X^{(i)}, y^{(i)})$. In order to fit a multiple linear regression model regressing $y$ on $X$ using this data we return to the mean square error.

$$
MSE = \frac{1}{n} \sum_{i=1}^n \left(y^{(i)} - \hat{y}^{(i)}\right)^2 = \frac{1}{n} \sum_{i=1}^n \left(y^{(i)} - X^{(i)} \hat{\beta}\right)^2,
$$

we can rewrite this using some linear algebra as:

$$
MSE = \frac{1}{n}\left(y - X\hat{\beta} \right)^T\left(y-X\hat{\beta} \right) = \frac{1}{n}\left( y^T y - \hat{\beta}^T X^T y - y^T X \hat{\beta} + \hat{\beta}^T X^T X \hat{\beta}\right). 
$$

When you take the derivative with respect to $\hat{\beta}$ and set it equal to $0$ gives the following:

$$
X^T X \hat{\beta} - X^T y = 0, \text{ and so } \hat{\beta} = (X^T X)^{-1}X^T y.
$$

This is the <i>ordinary least squares</i> estimate of the coefficient vector $\beta$. Note that this formula is also sometimes called the <i>normal equation</i>.

### Back to baseball

We will demonstrate how to fit this model in `numpy` and `sklearn` with the baseball example we looked at in the previous notebook.

In [2]:
## Note this works on Mac and Linux,
## you may need to change the slash directions if
## you are running a Windows machine
baseball = pd.read_csv("../../../data/baseball.csv")

In [3]:
baseball.head(2)

,teamID,yearID,lgID,G,W,L,R,RA
0,ANA,2001,AL,162,75,87,691,730
1,ARI,2001,NL,162,92,70,818,677


In [4]:
## import train_test_split
from sklearn.model_selection import train_test_split

In [5]:
## make the train test split here
## Note a slight difference, we have to use .copy()
## for pandas dataframes
bb_train, bb_test = train_test_split(baseball.copy(),
                                        shuffle = True,
                                        random_state = 440,
                                        test_size=.2)

The multiple linear regression model that we will fit is:

$$
\texttt{W} = \beta_0 + \beta_1 \texttt{R} + \beta_2 \texttt{RA} + \epsilon.
$$

In [6]:
## remember X needs a column of 1s
X_train = np.ones((len(bb_train), 3))

X_train[:,1:] = bb_train[['R', 'RA']].values
y_train = bb_train.W.values

In [7]:
X_train

array([[  1., 667., 633.],
       [  1., 770., 580.],
       [  1., 718., 700.],
       ...,
       [  1., 657., 858.],
       [  1., 678., 831.],
       [  1., 513., 646.]])

In [8]:
y_train

array([ 86, 104,  83,  85, 100,  97, 105,  90,  95,  69,  91,  70,  91,
        71,  83, 101,  75,  62,  69,  95,  85, 100,  93,  73,  56,  80,
        88,  78,  76,  62,  78,  89,  93,  83,  77,  68,  76,  78,  76,
        79,  97,  63,  82,  83,  81,  98,  96,  89,  83,  81,  86,  80,
        74,  75,  91,  77,  97, 103,  85,  88,  90,  81,  64,  68,  66,
        64,  63,  89,  71,  68,  92,  67,  86,  86,  83,  77,  84,  87,
        86,  85,  76,  66,  82,  83,  80,  76,  81,  83,  88,  98,  92,
        71,  75,  90,  78,  86,  75,  75,  84,  71,  92,  72,  96,  62,
        78,  79,  71,  71,  90,  96,  51,  69,  68,  67,  75,  63,  96,
        81,  93,  97,  83, 108,  88,  96,  73,  85,  90,  82,  88,  87,
        92,  74,  67,  72,  91,  80,  75,  72,  89,  88,  87,  68,  66,
        85,  70,  82,  92,  92,  75,  67,  77,  68,  75,  65,  85,  73,
        86,  96,  73,  65,  62,  80,  67,  98,  95,  97,  71,  82,  79,
        93,  75,  71,  80,  69,  93,  83,  85,  55,  81,  75,  8

In [10]:
## Calculate the normal equation
## note we'll use the linalg subpackage of numpy a lot
## here is a link to the documentation
## https://numpy.org/doc/stable/reference/routines.linalg.html
beta_hat = np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y_train)

In [11]:
# looking at beta_hat
print("beta_0_hat =", beta_hat[0])
print("beta_1_hat =", beta_hat[1])
print("beta_2_hat =", beta_hat[2])

beta_0_hat = 84.08213739723357
beta_1_hat = 0.09718334537049489
beta_2_hat = -0.10132296424142645


In [12]:
## Make the predictions 
y_pred_numpy = beta_hat[0] + beta_hat[1] * X_train[:,1] + beta_hat[2] * X_train[:,2]

In [13]:
## calculate the mse
print("the training mse is", np.sum(np.power(y_train-y_pred_numpy, 2))/len(y_train))

the training mse is 16.945434114275745


### Using `sklearn`

We will end this notebook by showing how simple it is to use `sklearn` to fit this model. In fact you already know how to do it!

In [14]:
## import the LinearRegression object
from sklearn.linear_model import LinearRegression

In [15]:
## Make the model object
## notice we have to us fit_intercept = False
## because X_train has a column of 1s
reg = LinearRegression(copy_X=True, fit_intercept=False)

## Fit the model object
## note I do NOT have to use reshape here
## because X_train is a 2D np.array
reg.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

In [16]:
## look at coef
reg.coef_

array([84.0821374 ,  0.09718335, -0.10132296])

In [17]:
## Make a prediction
y_pred_sklearn = reg.predict(X_train)

In [18]:
## calculate the mse
print("the mse is",np.sum(np.power(y_train-y_pred_sklearn, 2))/len(y_train))

the mse is 16.945434114275745


That's it!

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)